In [18]:
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import dash

In [3]:
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv"
df = pd.read_csv(url)

In [4]:
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [5]:
rows_with_missing = df.isnull().any(axis=1).sum()
print(rows_with_missing)

11


In [6]:
11/344


0.03197674418604651

In [7]:
df.dropna(inplace=True)

tylko 3% wierszy ma w sobie braki danych, zatem one nie wpłyną nam za bardzo  na jakość wyników

In [8]:
def filter_data(data, sex, species):
    # data = pd.DataFrame.from_dict(raw_data)

    if sex != "All":
        # data = data[data["sex"] == sex]
        data = data.loc[df['sex'] == sex]

    if species != "All":
        # data = data[data["species"] == species]
        data = data.loc[df['species'] == species]
    
    # print(data)

    return data


In [9]:
df.island.unique()

array(['Torgersen', 'Biscoe', 'Dream'], dtype=object)

In [10]:
color_map = {
     "Torgersen": "#7e6c6c",
    "Biscoe": "#ffa9a3",
    "Dream": "#b9e6ff"

}

In [11]:
def update_charts(filtered_data, param):
    
    # data = pd.DataFrame.from_dict(filtered_data)
    data = filtered_data
    data['island'] = data['island'].astype('category')

    group_df = data.groupby("island")[param].mean().reset_index().sort_values(by=param)
    group_df_dist = [data[data['island'] == island][param].dropna() for island in data['island'].unique()]


#     $cinereous: rgba(126, 108, 108, 1);
# $light-red: rgba(248, 117, 117, 1);
# $melon: rgba(255, 169, 163, 1);
# $uranian-blue: rgba(185, 230, 255, 1);

    # Wykres średnich wartości na wyspę
    mean_chart = px.bar(
        group_df,
        x="island", y=param,
        title="Średnia wartość wybranego parametru na wyspę",
        labels={"island": "Wyspa", param: "Średnia wartość"},
        template="plotly_white",
        color="island",
        color_discrete_map={
                "Torgersen": "#7e6c6c",
                "Biscoe": "#ffa9a3",
                "Dream": "#b9e6ff"}
        # showlegend=False
    )
    mean_chart.update_traces(showlegend = False)
    mean_chart.update_layout(title_text="Średnia wartość wybranego parametru na wyspę", title_x=0.5)


    distribution_chart = px.histogram(
        filtered_data,
        x=param,
        title="Rozkład wybranego parametru (histogram gęstości)",
        labels={param: "Wartość parametru"},
        nbins=30,
        template="plotly_white",
        color = 'island',
        color_discrete_map={
                "Torgersen": "#7e6c6c",
                "Biscoe": "#ffa9a3",
                "Dream": "#b9e6ff"},
       opacity=0.6,
       histnorm='density'    

    )
    distribution_chart.update_traces(showlegend = True)
    distribution_chart.update_layout(
        legend=dict(
            title=dict(
                text="Wyspy"
            )
        ),
        title_text='Rozkładu wybranego parametru', 
        title_x=0.5
    )


    return mean_chart, distribution_chart


In [12]:
filtered_data = filter_data(df, sex='All', species='All')

In [13]:
filtered_data

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,FEMALE
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [14]:
charts = update_charts(filtered_data, "bill_length_mm")




C:\Users\kkarb\AppData\Local\Temp\ipykernel_17404\100074652.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_df = data.groupby("island")[param].mean().reset_index().sort_values(by=param)


In [15]:
charts[0].show()

In [16]:
charts[1].show()

In [17]:
print(pd.__version__)

2.2.2


In [19]:
print(dash.__version__)

2.17.1
